In [ ]:
#import standard libraries

import numpy as np
import matplotlib.pyplot as plt
import tidynamics as td
from scipy.optimize import curve_fit
from scipy.integrate import cumtrapz

In [ ]:
job_name = '50507548'                          # '50507546', '50507547', '50507548'
data_dir = '/Volumes/work/genius/scratch/'

In [ ]:
def exp_fit(t, p0, p1):
    return p0 * (1-np.exp(-p1*t))

In [ ]:
#visc_gk = open('visc_gk.txt', 'a')

time = np.arange(0.0, 500000*0.005*10, 10*0.005)
samples = np.arange(len(time), 0, -1)
cutoff = 5000
limit = 2.0

for i in range(0,40):
    j = i+1
    
    with open('./params_dpd.csv') as f:
        line = f.readlines()[j]
        line = line.split(',')
        
    rho, a_ij = float(line[0]), float(line[1])
    #print(rho,a_ij)
    
    l = (10000/rho) ** (1/3)
    vol = 10000/rho
    eta_theo = 2*np.pi*4.5*rho**2/1575
    
    #Here we do the calculation by hand using tidynamics
    
    kin_stress = -1.0 * np.genfromtxt(
        data_dir + 'dpd_visc_' + job_name + '[' + str(j) +']/kin_stress.txt')
    
    old_stress = -1.0 * np.genfromtxt(
        data_dir + 'dpd_visc_' + job_name + '[' + str(j) +']/old_stress.txt')
    
    dpd_stress = -1.0 * np.genfromtxt(
        data_dir + 'dpd_visc_' + job_name + '[' + str(j) +']/dpd_stress.txt')
    
    kin_stress_acf = td.acf(kin_stress)
    old_stress_acf = td.acf(old_stress)
    dpd_stress_acf = td.acf(dpd_stress)
    
    kin_stress_int = cumtrapz(kin_stress_acf, x = time, initial = 0)
    old_stress_int = cumtrapz(old_stress_acf, x = time, initial = 0)
    dpd_stress_int = cumtrapz(dpd_stress_acf, x = time, initial = 0)
    
    temp = old_stress_int
    
    kin_visc, pcov = curve_fit(exp_fit, time[:cutoff], kin_stress_int[:cutoff], sigma=1/np.sqrt(samples[:cutoff]))
    old_visc, pcov = curve_fit(exp_fit, time[:cutoff], old_stress_int[:cutoff], sigma=1/np.sqrt(samples[:cutoff]))
    dpd_visc, pcov = curve_fit(exp_fit, time[:cutoff], dpd_stress_int[:cutoff], sigma=1/np.sqrt(samples[:cutoff]))
    
    plt.figure(figsize=(17,17*2/3))
    
    ax1 = plt.subplot(231)
    ax1.set_title('Kinetic stress at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax1.semilogx(time, kin_stress_int, 'o', markersize='3', color='tab:green', label = 'ACF tidynamics')
    ax1.semilogx(time, exp_fit(time, *kin_visc), '--', color='tab:green', label = 'Fit to ACF tidynamics')
    ax1.axvline(limit, color='tab:red')
    ax1.set_xlim(time[1], 1e2)
    ax1.set_ylim(0, 1.5*kin_visc[0])
    ax1.legend(loc='upper left')
    ax1.set_xlabel('time')
    ax1.set_ylabel('Integral of stress acf')
    ax1.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    
    idx = np.searchsorted(time, limit)
    ax1.axhline(kin_stress_int[idx], color='tab:red')
    
    ax1.text(0.9, 0.1, "Kinetic viscosity own correlation:\n{0:.3f}".format(kin_stress_int[idx]*vol),
             horizontalalignment='right',
             verticalalignment='center',
             transform = ax1.transAxes)    
    
    plt.tight_layout()
    
    ax4 = plt.subplot(234)
    ax4.set_title('Decay of kinetic stress ACF at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax4.loglog(time, kin_stress_acf, 'o', markersize='3', color='tab:green', label = 'ACF tidynamics')
    ax4.loglog(time[:100], time[:100]**(-3/2)*1e-4, '--', color='tab:red', label = 'powerlaw -3/2')
    ax4.loglog(time[:100], time[:100]**(-5/2)*1e-4, '--', color='tab:orange', label = 'powerlaw -5/2')
    ax4.loglog(time[:100], time[:100]**(-7/2)*1e-4, '--', color='tab:purple', label = 'powerlaw -7/2')
    ax4.axvline(limit, color='tab:red')
    ax4.set_xlim(time[1], 1e2)
    ax4.set_xlabel('time')
    ax4.set_ylabel('ACF')
    ax4.legend(loc='lower left')
    ax4.set_ylim(1e-7,1e-2)
    
    ax2 = plt.subplot(232)
    ax2.set_title('Conservative stress at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax2.semilogx(time, old_stress_int, 'o', markersize='3', color='tab:green', label = 'ACF tidynamics')
    ax2.semilogx(time, exp_fit(time, *old_visc), '--', color='tab:green', label = 'Fit to ACF tidynamics')
    ax2.axvline(limit, color='tab:red')
    ax2.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax2.set_xlim(time[1], 1e2)
    ax2.set_xlabel('time')
    
    ax5 = plt.subplot(235)
    ax5.set_title('Decay of old stress ACF at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax5.loglog(time, old_stress_acf, 'o', markersize='3', color='tab:green', label = 'ACF tidynamics')
    ax5.loglog(time[:100], time[:100]**(-3/2)*1e-3, '--', color='tab:red', label = 'powerlaw -3/2')
    ax5.loglog(time[:100], time[:100]**(-5/2)*1e-3, '--', color='tab:orange', label = 'powerlaw -5/2')
    ax5.loglog(time[:100], time[:100]**(-7/2)*1e-3, '--', color='tab:purple', label = 'powerlaw -7/2')
    ax5.axvline(limit, color='tab:red')
    ax5.set_xlabel('time')
    ax5.set_ylim(1e-7,1e-2)
    ax5.set_xlim(time[1], 1e2)
    
    ax3 = plt.subplot(233)
    ax3.set_title('Dissipative stress at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax3.semilogx(time, dpd_stress_int, 'o', markersize='3', color='tab:green', label = 'ACF tidynamics')
    #ax3.semilogx(time, exp_fit(time, *dpd_visc), '--', color='tab:green', label = 'Fit to ACF tidynamics')
    #ax3.axvline(limit, color='tab:red')
    ax3.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax3.set_xlim(1e-3, 1e2)
    ax3.set_xlabel('time')
    
    ax6 = plt.subplot(236)
    ax6.set_title('Decay of dpd stress ACF at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax6.loglog(time, dpd_stress_acf, 'o', markersize='3', color='tab:green', label = 'ACF tidynamics')
    ax6.loglog(time[:100], time[:100]**(-3/2)*1e-7, '--', color='tab:red', label = 'powerlaw -3/2')
    ax6.loglog(time[:100], time[:100]**(-5/2)*1e-7, '--', color='tab:orange', label = 'powerlaw -5/2')
    ax6.loglog(time[:100], time[:100]**(-7/2)*1e-7, '--', color='tab:purple', label = 'powerlaw -7/2')
    #ax6.axvline(limit, color='tab:red')
    ax6.set_xlabel('time')
    ax6.set_xlim(1e-3, 1e2)
    
    # Here we use the correlated data from the Espresso correlator
    # The kinetic component can't be extracted here
    
    old_stress = np.load(
        data_dir + 'dpd_visc_' + job_name + '[' + str(j) +']/dpd_sample_old_stress_acf.npy')
    
    old_stress_avg = (old_stress[:,3] + old_stress[:,4] + old_stress[:,7]) / 3.
    
    old_stress_int = cumtrapz(old_stress_avg, x = old_stress[:,0], initial = 0)
    old_visc, pcov = curve_fit(exp_fit, old_stress[:100,0], old_stress_int[:100], sigma=1/np.sqrt(old_stress[:100,1]))
    
    dpd_stress = np.load(
        data_dir + 'dpd_visc_' + job_name + '[' + str(j) +']/dpd_sample_dpd_stress_acf.npy')
    
    dpd_stress_avg = (dpd_stress[:,3] + dpd_stress[:,4] + dpd_stress[:,7]) / 3.
    
    dpd_stress_int = cumtrapz(dpd_stress_avg, x = dpd_stress[:,0], initial = 0)
    dpd_visc, pcov = curve_fit(exp_fit, dpd_stress[:65,0], dpd_stress_int[:65], sigma=1/np.sqrt(dpd_stress[:65,1]))
    
    ax2 = plt.subplot(232)
    ax2.semilogx(old_stress[:,0], old_stress_int, 'o', markersize='3', color='tab:blue', label = 'ACF ESPResSo')
    ax2.semilogx(old_stress[:,0], exp_fit(old_stress[:,0], *old_visc), '--', color='tab:blue', label = 'Fit to ACF ESPResSo')
    ax2.set_ylim(0, 1.5*old_visc[0])
    ax2.legend(loc='upper left')
    
    idx = np.searchsorted(old_stress[:,0], limit)
    ax2.axhline(old_stress_int[idx], color='tab:red')
    
    ax2.text(0.9, 0.1, 
             "Conservative viscosity:\n{0:.3f}".format(old_stress_int[idx]*vol),
             horizontalalignment='right',
             verticalalignment='center',
             transform = ax2.transAxes)    
    
    plt.tight_layout()
    
    ax5 = plt.subplot(235)
    ax5.set_title('Decay of old stress ACF at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax5.loglog(old_stress[:,0], old_stress_avg, 'o', markersize='3', color='tab:blue', label = 'ACF ESPResSo')
    ax5.legend(loc='lower left')
    plt.tight_layout()
    
    ax3 = plt.subplot(233)
    ax3.semilogx(dpd_stress[:,0], dpd_stress_int, 'o', markersize='3', color='tab:blue', label = 'ACF ESPResSo')
    #ax3.semilogx(dpd_stress[:,0], exp_fit(dpd_stress[:,0], *dpd_visc), '--', color='tab:blue', label = 'Fit to ACF ESPResSo')
    ax3.axhline((2*np.pi*4.5*rho**2/1575)/vol, color='tab:olive')
    ax3.set_ylim(0, 1.5*dpd_visc[0])
    ax3.legend(loc='upper left')
    
    dpd_idx = 1
    ax3.axhline(dpd_stress_int[dpd_idx], color='tab:red')
    ax3.text(0.9, 0.1, 
             "Dissipative viscosity:\n{:.3f} \nTheoretical estimate:\n{:.3f}".format(dpd_stress_int[dpd_idx]*vol, eta_theo), 
             horizontalalignment='right',
             verticalalignment='center',
             transform = ax3.transAxes)    
    
    plt.tight_layout()
    
    ax6 = plt.subplot(236)
    ax6.set_title('Decay of DPD stress ACF at \n density=' + str(rho) + '\n rep force=' + str(a_ij))
    ax6.loglog(dpd_stress[:,0], dpd_stress_avg, 'o', markersize='3', color='tab:blue', label = 'ACF ESPResSo')
    ax6.legend(loc='lower left')
    ax6.set_ylim(1e-8,None)
    plt.tight_layout()
    
    #visc_gk.write('{}; {}; {}; {}; {}; {} \n'.format(rho, a_ij, kin_stress_int[idx]*vol, old_stress_int[idx]*vol, dpd_stress_int[dpd_idx]*vol, eta_theo))

#visc_gk.close()